# Семинар 1. Базовая предобработка текста, Bag-of-Words и TF‑IDF (Colab-friendly)

**Цель семинара:** научиться превращать «сырой» текст в численные признаки и собрать первый рабочий бейзлайн для классификации текстов.

> Формат: короткие объяснения → маленькие примеры → один «мини‑проект» в конце.  
> Если что-то ломается — это не вы, это жизнь 🙂 (ниже есть ячейки‑фолбэки).

---

## План
1. Мини‑праймер: текст в Python (строки, юникод, базовые операции)
2. Токенизация: split vs готовые токенизаторы
3. Стоп‑слова и пунктуация (и когда **не** надо их выкидывать)
4. Лемматизация и стемминг (mystem vs pymorphy3, Snowball)
5. Векторизация: Bag‑of‑Words, n‑граммы, символьные n‑граммы
6. TF‑IDF: интуиция + ручной расчёт на игрушечном примере + sklearn
7. Мини‑проект: классификация тональности твитов + таблица экспериментов
8. Регулярные выражения: чистка твитов + маленькие задачки

---

🧠 **Важно:** в прошлой лекции мы уже обсуждали «введение в NLP». Здесь мы держим вводную короткой и быстро переходим к практике.


In [ ]:
# === Установка зависимостей (Colab) ===
# Если вы не в Colab — эти команды всё равно обычно работают в Jupyter.

!pip -q install nltk pymorphy3 pymorphy3-dicts-ru pymystem3 scikit-learn pandas numpy

import re
import os
import math
import random
import numpy as np
import pandas as pd

# NLTK ресурсы
import nltk
nltk.download("punkt", quiet=True)
nltk.download("stopwords", quiet=True)

random.seed(42)
np.random.seed(42)

print("✅ Готово")


✅ Готово


## Пакеты, которыми мы пользуемся (один раз фиксируем)

- **nltk** — токенизация, стоп‑слова, простые стеммеры, n‑граммы.
- **pymorphy3** — быстрый морфоанализ/лемматизация русского (по словам).
- **pymystem3** — обёртка над *mystem* (лемматизация + попытка снятия омонимии по контексту).
- **scikit‑learn** — `CountVectorizer`, `TfidfVectorizer`, модели (логрег), метрики.
- **re** — регулярные выражения (чистка текста, извлечение паттернов).
- **pandas/numpy** — таблички и вычисления.

Дальше будем пользоваться ими по делу.


## 1) Мини‑праймер: текст в Python

Текст в Python — это **строка** (`str`) в **Unicode**.

Полезные операции, которые реально встречаются в NLP‑коде:
- `.lower()` — привести к нижнему регистру
- `.strip()` — убрать пробелы по краям
- `.replace(a, b)` — заменить подстроку
- `.split()` / `.splitlines()` — разбиение
- `f"..."` — f‑строки
- `r"..."` — *raw string* (удобно для regex: `r"\d+"`)

Посмотрим на маленьком примере.


In [ ]:
text = "Привет!  \nЭто строка №1.\nА это строка №2 :)"
print(text)

print("-----")
print("lower:", text.lower())
print("strip:", "   hello   ".strip())
print("replace:", "кошки, кошки".replace("кошки", "коты"))

print("-----")
print("splitlines:", text.splitlines())
print("split по пробелам:", "a  b   c".split())

# f-string
name = "Андрей"
print(f"Привет, {name}! Сегодня мы будем векторизовать текст 🔥")


Привет!  
Это строка №1.
А это строка №2 :)
-----
lower: привет!  
это строка №1.
а это строка №2 :)
strip: hello
replace: коты, коты
-----
splitlines: ['Привет!  ', 'Это строка №1.', 'А это строка №2 :)']
split по пробелам: ['a', 'b', 'c']
Привет, Андрей! Сегодня мы будем векторизовать текст 🔥


## 2) Токенизация

**Токенизировать** — значит разбить текст на токены (часто — слова, иногда — символы, эмодзи, хэштеги, и т. п.)

Самый наивный вариант: `split()` по пробелам.  
Проблема: он не отделяет пунктуацию, не понимает смайлики, сокращения и т. д.

На практике почти всегда используют готовые токенизаторы.


In [ ]:
import nltk
import warnings
warnings.filterwarnings("ignore")

# На новых версиях NLTK часто нужен именно punkt_tab
nltk.download("punkt_tab", quiet=True)

# На всякий случай (иногда встречается в окружениях/старых примерах)
nltk.download("punkt", quiet=True)

from nltk.tokenize import word_tokenize
from nltk import tokenize

example = "Но не каждый хочет что-то исправлять:("

print("split():", example.split())
print("word_tokenize():", word_tokenize(example, language="russian"))

wh_tok = tokenize.WhitespaceTokenizer()
print("Whitespace spans:", list(wh_tok.span_tokenize(example)))

print("Treebank (EN):", tokenize.TreebankWordTokenizer().tokenize("don't stop me"))


split(): ['Но', 'не', 'каждый', 'хочет', 'что-то', 'исправлять:(']
word_tokenize(): ['Но', 'не', 'каждый', 'хочет', 'что-то', 'исправлять', ':', '(']
Whitespace spans: [(0, 2), (3, 5), (6, 12), (13, 18), (19, 25), (26, 38)]
Treebank (EN): ['do', "n't", 'stop', 'me']


### 2.1 Токенизаторы в NLTK (быстрый обзор)

В `nltk.tokenize` есть разные токенизаторы под разные типы текста:
- `word_tokenize` — «универсальный» (на практике: удобно, но не идеален)
- `WordPunctTokenizer` — простая regex‑разбивка слов/пунктуации
- `TweetTokenizer` — полезен для соцсетей (эмодзи, хэштеги, повторы символов)
- `RegexpTokenizer` — вы сами задаёте regex, что считать токеном


In [ ]:
from nltk.tokenize import TweetTokenizer

tw = TweetTokenizer()
print(tw.tokenize(example))

# Частый паттерн: оставить только "словесные" токены
tokens_alpha = [w for w in word_tokenize(example, language="russian") if w.isalpha()]
print(tokens_alpha)


['Но', 'не', 'каждый', 'хочет', 'что-то', 'исправлять', ':(']
['Но', 'не', 'каждый', 'хочет', 'исправлять']


## 3) Стоп‑слова и пунктуация

**Стоп‑слова** — частые слова, которые часто мало помогают различать документы (например, «и», «в», «это»).  
Их часто убирают, как и пунктуацию.

⚠️ Но! Иногда пунктуация/смайлики — **сильный сигнал** (особенно в соцсетях).  
Мы ниже увидим пример, где один символ почти решает задачу 🙂


In [ ]:
from nltk.corpus import stopwords
from string import punctuation

ru_stop = set(stopwords.words("russian"))
en_stop = set(stopwords.words("english"))
punct = set(punctuation)

In [ ]:
len(en_stop)

198

In [ ]:


print("Пример RU стоп-слов:", list(sorted(ru_stop))[:20])
print("punctuation:", "".join(list(punctuation)[:15]), "...")

tokens = word_tokenize(example, language="russian")
noise = ru_stop | punct

clean_tokens = [t.lower() for t in tokens if t.lower() not in noise]
print("tokens:", tokens)
print("clean_tokens (без стоп-слов и пунктуации):", clean_tokens)


Пример RU стоп-слов: ['а', 'без', 'более', 'больше', 'будет', 'будто', 'бы', 'был', 'была', 'были', 'было', 'быть', 'в', 'вам', 'вас', 'вдруг', 'ведь', 'во', 'вот', 'впрочем']
punctuation: !"#$%&'()*+,-./ ...
tokens: ['Но', 'не', 'каждый', 'хочет', 'что-то', 'исправлять', ':', '(']
clean_tokens (без стоп-слов и пунктуации): ['каждый', 'хочет', 'что-то', 'исправлять']


## 4) Лемматизация и стемминг

### 4.1 Лемматизация
Лемматизация приводит слово к **нормальной форме** (*лемме*):
- «пью», «пил», «пьёт» → **«пить»**
- «кошки», «кошкой», «кошек» → **«кошка»**

Почему полезно:
1) меньше «раздуваем» словарь (форма слова ≠ новый признак)  
2) легче ловить стоп‑слова (они часто заданы в начальной форме)

Для русского часто используют **mystem** и **pymorphy3**.


In [ ]:
example = 'Современный русский язык насчитывает около 500 тысяч слов, включая литературную лексику, диалекты, профессиональные термины и жаргонизмы, по данным комплексного анализа Института Пушкина, озвученным в феврале 2026 года.'

In [ ]:
example

'Современный русский язык насчитывает около 500 тысяч слов, включая литературную лексику, диалекты, профессиональные термины и жаргонизмы, по данным комплексного анализа Института Пушкина, озвученным в феврале 2026 года.'

In [ ]:
# --- Mystem (контекстный анализ; может быть медленным) ---
try:
    from pymystem3 import Mystem
    mystem = Mystem()
    print("mystem OK ✅")
    print(mystem.lemmatize(example))
except Exception as e:
    print("mystem не завёлся 😿")
    print("Причина:", repr(e))
    print("Не страшно: ниже будет pymorphy3, он обычно работает стабильно в Colab.")


mystem OK ✅
['современный', ' ', 'русский', ' ', 'язык', ' ', 'насчитывать', ' ', 'около', ' ', '500', ' ', 'тысяча', ' ', 'слово', ', ', 'включая', ' ', 'литературный', ' ', 'лексика', ', ', 'диалект', ', ', 'профессиональный', ' ', 'термин', ' ', 'и', ' ', 'жаргонизм', ', ', 'по', ' ', 'данные', ' ', 'комплексный', ' ', 'анализ', ' ', 'институт', ' ', 'пушкин', ', ', 'озвучивать', ' ', 'в', ' ', 'февраль', ' ', '2026', ' ', 'год', '.', '\n']


In [ ]:
# --- Pymorphy3 (быстро, но по одному слову, без контекстной дизамбигуации) ---
from pymorphy3 import MorphAnalyzer
morph = MorphAnalyzer()

word = "Современный русский язык насчитывает"
ana = morph.parse(word)
print("Разборы:", ana)
print("Лемма (normal_form):", ana[0].normal_form)
print("Тег:", ana[0].tag)


Разборы: [Parse(word='современный русский язык насчитывает', tag=OpencorporaTag('VERB,impf,tran sing,3per,pres,indc'), normal_form='современный русский язык насчитывать', score=0.9999999999999999, methods_stack=((FakeDictionary(), 'современный русский язык насчитывает', 215, 5), (KnownSuffixAnalyzer(min_word_length=4, score_multiplier=0.5), 'ывает')))]
Лемма (normal_form): современный русский язык насчитывать
Тег: VERB,impf,tran sing,3per,pres,indc


Разборы: [Parse(word='хочет', tag=OpencorporaTag('VERB,impf,tran sing,3per,pres,indc'), normal_form='хотеть', score=1.0, methods_stack=((DictionaryAnalyzer(), 'хочет', 3136, 5),))]


**Омонимия:** одно и то же написание — разные значения/части речи.  
mystem *пытается* учитывать контекст, pymorphy3 — нет (ему даём слово отдельно).


In [ ]:
homonym1 = "За время обучения я прослушал больше сорока курсов."
homonym2 = "Сорока своровала блестящее украшение со стола."

try:
    print("mystem #1:", mystem.analyze(homonym1)[-5])
    print("mystem #2:", mystem.analyze(homonym2)[0])
except Exception:
    print("mystem недоступен — пропускаем этот блок.")

# pymorphy3: без контекста слово "сорока" обычно будет как существительное
print("pymorphy3:", morph.parse("сорока")[0])


mystem #1: {'analysis': [{'lex': 'сорок', 'wt': 0.8710292664, 'gr': 'NUM=(пр|дат|род|твор)'}], 'text': 'сорока'}
mystem #2: {'analysis': [{'lex': 'сорока', 'wt': 0.1210970041, 'gr': 'S,жен,од=им,ед'}], 'text': 'Сорока'}
pymorphy3: Parse(word='сорока', tag=OpencorporaTag('NUMR gent'), normal_form='сорок', score=0.68, methods_stack=((DictionaryAnalyzer(), 'сорока', 2920, 1),))


### 4.2 Стемминг
Стемминг «обрезает» аффиксы по правилам языка и не обязан выдавать реальное слово:
- «исправлять» → «исправля»
Это быстрее и проще, но может давать «странные» стемы.

В NLTK есть SnowballStemmer для разных языков.


In [ ]:
from nltk.stem.snowball import SnowballStemmer

ru_stemmer = SnowballStemmer("russian")
tokenized_example = word_tokenize(example, language="russian")
stemmed = [ru_stemmer.stem(w.lower()) for w in tokenized_example if w.isalpha()]

print("Токены:", [w for w in tokenized_example if w.isalpha()])
print("Стемы:", stemmed)

en_text = "Whenever you feel like criticizing any one, just remember that all the people in this world haven't had the advantages that you've had."
en_tokens = [w for w in word_tokenize(en_text, language="english") if w.isalpha()]
en_stemmer = SnowballStemmer("english")
print("EN stems:", " ".join(en_stemmer.stem(w.lower()) for w in en_tokens[:20]), "...")


Токены: ['Современный', 'русский', 'язык', 'насчитывает', 'около', 'тысяч', 'слов', 'включая', 'литературную', 'лексику', 'диалекты', 'профессиональные', 'термины', 'и', 'жаргонизмы', 'по', 'данным', 'комплексного', 'анализа', 'Института', 'Пушкина', 'озвученным', 'в', 'феврале', 'года']
Стемы: ['современ', 'русск', 'язык', 'насчитыва', 'окол', 'тысяч', 'слов', 'включ', 'литературн', 'лексик', 'диалект', 'профессиональн', 'термин', 'и', 'жаргонизм', 'по', 'дан', 'комплексн', 'анализ', 'институт', 'пушкин', 'озвучен', 'в', 'феврал', 'год']
EN stems: whenev you feel like critic ani one just rememb that all the peopl in this world have had the advantag ...


## 5) Векторизация: Bag‑of‑Words и n‑граммы

Модели ML хотят числа, а не слова.  
**Bag‑of‑Words (BoW)** — самый простой способ: считаем, сколько раз каждое слово из словаря встретилось в документе.

Словарь строим по корпусу (набору текстов), дальше каждый текст — это вектор длины `|V|`.


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

texts = [
    "I like my cat.",
    "My cat is the most perfect cat.",
    "is this cat or is this bread My?",
]

# Быстрая очистка: оставим только слова (alpha), чтобы не мешала пунктуация
texts_clean = [" ".join([w for w in word_tokenize(t, language="english") if w.isalpha()]) for t in texts]
print(texts_clean)


['I like my cat', 'My cat is the most perfect cat', 'is this cat or is this bread My']


In [ ]:

cnt_vec = CountVectorizer()
X = cnt_vec.fit_transform(texts_clean)


In [ ]:

print("Словарь:", cnt_vec.vocabulary_)
df_bow = pd.DataFrame(X.toarray(), columns=cnt_vec.get_feature_names_out())
df_bow


Словарь: {'like': 3, 'my': 5, 'cat': 1, 'is': 2, 'the': 8, 'most': 4, 'perfect': 7, 'this': 9, 'or': 6, 'bread': 0}


,bread,cat,is,like,most,my,or,perfect,the,this
0,0,1,0,1,0,1,0,0,0,0
1,0,2,1,0,1,1,0,1,1,0
2,1,1,2,0,0,1,1,0,0,2


### 5.1 n‑граммы

**n‑грамма** — последовательность из `n` токенов.
- 1‑граммы: `("это")`
- 2‑граммы: `("не", "каждый")`
- 3‑граммы: `("если", "б", "мне")`

В `CountVectorizer` это включается параметром `ngram_range`.


In [ ]:
from nltk import ngrams

sent = "Если б мне платили каждый раз".split()
print("униграммы:", list(ngrams(sent, 1)))
print("биграммы:", list(ngrams(sent, 2)))
print("триграммы:", list(ngrams(sent, 3)))

# Векторизация биграммами
vec_bi = CountVectorizer(ngram_range=(1, 1))
X_bi = vec_bi.fit_transform(texts_clean)
print("bigrams:", vec_bi.get_feature_names_out())
pd.DataFrame(X_bi.toarray(), columns=vec_bi.get_feature_names_out())


униграммы: [('Если',), ('б',), ('мне',), ('платили',), ('каждый',), ('раз',)]
биграммы: [('Если', 'б'), ('б', 'мне'), ('мне', 'платили'), ('платили', 'каждый'), ('каждый', 'раз')]
триграммы: [('Если', 'б', 'мне'), ('б', 'мне', 'платили'), ('мне', 'платили', 'каждый'), ('платили', 'каждый', 'раз')]
bigrams: ['bread' 'cat' 'is' 'like' 'most' 'my' 'or' 'perfect' 'the' 'this']


,bread,cat,is,like,most,my,or,perfect,the,this
0,0,1,0,1,0,1,0,0,0,0
1,0,2,1,0,1,1,0,1,1,0
2,1,1,2,0,0,1,1,0,0,2


### 5.2 Символьные n‑граммы

Иногда полезно смотреть не на слова, а на последовательности символов (например, определение языка, «шумные» тексты, опечатки).

В `CountVectorizer` включается через `analyzer="char"`.


In [ ]:
vec_char = CountVectorizer(analyzer="char", ngram_range=(3, 5), min_df=1)
X_char = vec_char.fit_transform(["кот", "коты", "котик", "собака"])
print("Пример char-фичей:", vec_char.get_feature_names_out()[:15])
print("shape:", X_char.shape)


Пример char-фичей: ['ака' 'бак' 'бака' 'кот' 'коти' 'котик' 'коты' 'оба' 'обак' 'обака' 'оти'
 'отик' 'оты' 'соб' 'соба']
shape: (4, 17)


---

## 6) TF-IDF: интуиция и расчёт

BoW считает «частоты», но:

* слово, частое **в одном** документе и редкое **в остальных**, может быть очень информативным;
* слово, встречающееся «везде» (например, *the*, *и*) — плохо различает документы.

**TF-IDF** даёт высокий вес словам, которые:

* часто встречаются в документе (**TF**),
* но редко встречаются в корпусе (**IDF**).

---

### Формулы

$$
TF(t, d) = \frac{\mathrm{count}(t, d)}{|d|}
$$

где:

* $t$ — термин (слово),
* $d$ — документ,
* $|d|$ — общее число слов в документе.

---

$$
IDF(t) = \log \frac{N}{df(t)}
$$

где:

* $N$ — число документов,
* $df(t)$ — число документов, где встречается термин $t$.

---

$$
TFIDF(t, d) = TF(t, d)\cdot IDF(t)
$$

---

⚙️ В `sklearn` по умолчанию используется сглаживание:

$$
IDF(t) = \log \frac{1 + N}{1 + df(t)} + 1
$$

Это делается, чтобы:

* избежать деления на ноль,
* не получать нулевые веса,
* стабилизировать значения.

---

In [ ]:
toy_docs = [
    "the cat sat on mat",
    "the cat ate fish",
    "the dog ate fish",
]

# Токены
docs_tok = [d.split() for d in toy_docs]
vocab = sorted(set(w for d in docs_tok for w in d))
N = len(docs_tok)

def tf(word, doc):
    return doc.count(word) / len(doc)

def df(word, docs):
    return sum(1 for d in docs if word in d)

def idf_sklearn_style(word, docs):
    # как в sklearn (сглаживание)
    return math.log((1 + N) / (1 + df(word, docs))) + 1

# Табличка TF-IDF по sklearn-style формуле
rows = []
for i, doc in enumerate(docs_tok):
    for w in vocab:
        rows.append({
            "doc": i,
            "word": w,
            "tf": tf(w, doc),
            "df": df(w, docs_tok),
            "idf": idf_sklearn_style(w, docs_tok),
            "tfidf": tf(w, doc) * idf_sklearn_style(w, docs_tok),
        })
df_tfidf = pd.DataFrame(rows)

# Показать самые большие веса в каждом документе
df_tfidf.sort_values(["doc", "tfidf"], ascending=[True, False]).groupby("doc").head(5)


,doc,word,tf,df,idf,tfidf
4,0,mat,0.20,1,1.693147,0.338629
5,0,on,0.20,1,1.693147,0.338629
6,0,sat,0.20,1,1.693147,0.338629
1,0,cat,0.20,2,1.287682,0.257536
7,0,the,0.20,3,1.000000,0.200000
8,1,ate,0.25,2,1.287682,0.321921
9,1,cat,0.25,2,1.287682,0.321921
11,1,fish,0.25,2,1.287682,0.321921
15,1,the,0.25,3,1.000000,0.250000
10,1,dog,0.00,1,1.693147,0.000000


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vec = TfidfVectorizer()
X = tfidf_vec.fit_transform(toy_docs)

print("vocab:", tfidf_vec.get_feature_names_out())
pd.DataFrame(X.toarray(), columns=tfidf_vec.get_feature_names_out())


vocab: ['ate' 'cat' 'dog' 'fish' 'mat' 'on' 'sat' 'the']


,ate,cat,dog,fish,mat,on,sat,the
0,0.000000,0.38377,0.000000,0.000000,0.504611,0.504611,0.504611,0.298032
1,0.526820,0.52682,0.000000,0.526820,0.000000,0.000000,0.000000,0.409123
2,0.480458,0.00000,0.631745,0.480458,0.000000,0.000000,0.000000,0.373119


**Что поменялось по сравнению с BoW?**  
Теперь значения — не «количество», а **веса**: редкие, но характерные слова получают больший вклад.

На игрушечном примере заметно, что `mat` будет важно только в первом документе, а `fish` — во 2‑3.


## 7) Мини‑проект: классификация твитов по тональности

Задача: по тексту твита предсказать метку **positive/negative**.

Скачаем датасет (два файла: positive/negative).  
Если вы в Colab — `wget` почти всегда работает. Если нет — ниже есть Python‑фолбэк.


In [ ]:
import pathlib

def download(url, out_path):
    # Скачивает файл по URL. Работает и в Colab, и локально.
    out_path = pathlib.Path(out_path)
    out_path.parent.mkdir(parents=True, exist_ok=True)

    if out_path.exists() and out_path.stat().st_size > 0:
        print(f"✔ Уже скачано: {out_path}")
        return

    # 1) попробуем wget (быстро)
    try:
        !wget -q -O "{out_path}" "{url}"
        if out_path.exists() and out_path.stat().st_size > 0:
            print(f"✅ Скачано wget: {out_path} ({out_path.stat().st_size/1e6:.1f} MB)")
            return
    except Exception:
        pass

    # 2) фолбэк: requests
    import requests
    print("wget не сработал, пробуем requests...")
    r = requests.get(url, stream=True, timeout=60, allow_redirects=True)
    r.raise_for_status()
    with open(out_path, "wb") as f:
        for chunk in r.iter_content(chunk_size=1<<20):
            if chunk:
                f.write(chunk)
    print(f"✅ Скачано requests: {out_path} ({out_path.stat().st_size/1e6:.1f} MB)")

pos_url = "https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=1"
neg_url = "https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv?dl=1"

download(pos_url, "positive.csv")
download(neg_url, "negative.csv")


✔ Уже скачано: positive.csv
✔ Уже скачано: negative.csv


In [ ]:
# Читаем данные
positive = pd.read_csv("positive.csv", sep=";", usecols=[3], names=["text"])
positive["label"] = "positive"

negative = pd.read_csv("negative.csv", sep=";", usecols=[3], names=["text"])
negative["label"] = "negative"

df = pd.concat([positive, negative], ignore_index=True)
print("shape:", df.shape)
df.sample(5, random_state=42)


shape: (226834, 2)


,text,label
215081,мыс на меня обиделась:(\nя ей даже ничего не с...,negative
220754,"аааааааааааааааааааа,не хочу на работу :(",negative
60463,"У меня какой-то особенный вид ушей! :D, некото...",positive
213263,@simonovkon он неплохой человек в жизни. Я ра...,negative
113055,"RT @Darina_Lo: Домааааа\nЕхали на такси, пели ...",positive


In [ ]:
# Быстрый EDA
df["len"] = df["text"].astype(str).str.len()
print(df["label"].value_counts())

print("\nДлины (символы):")
print(df.groupby("label")["len"].describe()[["mean","std","min","50%","max"]])

# Примеры
for lbl in ["positive", "negative"]:
    print("\n---", lbl, "---")
    for t in df[df["label"]==lbl]["text"].sample(3, random_state=1).tolist():
        print("-", t[:150], "...")


label
positive    114911
negative    111923
Name: count, dtype: int64

Длины (символы):
              mean        std   min   50%    max
label                                           
negative  76.95052  28.800394   8.0  69.0  189.0
positive  85.20059  26.600490  21.0  79.0  179.0

--- positive ---
- @karin_karin_b вспомнил, что ты любишь котиков и решил запостить несколько своих фоток ^_^ http://t.co/gqKlDBNISo ...
- @Ksu_Lushnikova вдруг потом посмотрят, а записи нет) не хочу обидеть)) :D хоть мне и не нравятся записи такие, и человек мне никто, но жалко ...
- RT @Shady_Kate: @Arinorik Ооо, выздоравливай, Пивзушка.
Это хорошо, что настроение хорошее:D ...

--- negative ---
- @Flyulia1 нууу так не интересно:( надо придумывать новый план ...
- Очень люблю спать :( И эта разлука каждое утро убивает меня. ...
- Я готова делать все что угодно-мыть пол,стирать,красить ногти в сотый раз,пересаживать цветы, лишь бы не учить психологию( ...


### 7.1 Предобработка твитов (минимальная)

Для начала сделаем **минимальную** чистку:
- убрать URL
- убрать упоминания `@user`
- привести к нижнему регистру

Хэштеги и смайлики оставим — это часть смысла.


In [ ]:
URL_RE = re.compile(r"http\S+|www\.\S+")
MENTION_RE = re.compile(r"@\w+")
MULTISPACE_RE = re.compile(r"\s+")

def clean_tweet(text: str) -> str:
    text = str(text)
    text = text.replace("\n", " ")
    text = URL_RE.sub(" ", text)
    text = MENTION_RE.sub(" ", text)
    text = text.lower()
    text = MULTISPACE_RE.sub(" ", text).strip()
    return text

df["text_clean"] = df["text"].map(clean_tweet)
df[["text", "text_clean", "label"]].sample(5, random_state=0)


,text,text_clean,label
109500,RT @pydybeheqixo: Завтра я улетаю в Москву. А ...,rt : завтра я улетаю в москву. а послезавтра с...,positive
127808,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,rt : кто ненавидит пробки ретвит((((( rt,negative
193322,Почему? Почему у меня осталось так мало времен...,почему? почему у меня осталось так мало времен...,negative
202883,"Я уже вижу, как дети будут просматривать ваши ...","я уже вижу, как дети будут просматривать ваши ...",negative
172745,"@holiday_ann сам предмет маразм( и Аньч, прост...","сам предмет маразм( и аньч, прости, что н смог...",negative


### 7.2 Train/test split

Чтобы результаты были честными — разделим данные на train/test.


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df["text_clean"], df["label"], test_size=0.25, random_state=13, stratify=df["label"]
)

print("train:", X_train.shape, "test:", X_test.shape)


train: (170125,) test: (56709,)


### 7.3 Бейзлайн: логрег на BoW‑униграммах

Начнём с самого простого.


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MaxAbsScaler
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score
from scipy.sparse import csr_matrix

def evaluate_predictions(y_true, y_pred, y_proba=None):
    metrics = {
        "accuracy": accuracy_score(y_true, y_pred),
        "f1_macro": f1_score(y_true, y_pred, average="macro"),
        "f1_pos": f1_score(y_true, y_pred, pos_label="positive"),
    }
    if y_proba is not None:
        # roc_auc для бинарной классификации по вероятности positive
        y_bin = (pd.Series(y_true) == "positive").astype(int).values
        metrics["roc_auc"] = roc_auc_score(y_bin, y_proba)
    return metrics

def train_logreg(Xtr, Xte, ytr, yte, vec, use_scaler=True, max_iter=300):
    # Векторизация
    Xtr_vec = vec.fit_transform(Xtr)
    Xte_vec = vec.transform(Xte)

    if use_scaler:
        scaler = MaxAbsScaler()
        Xtr_vec = scaler.fit_transform(Xtr_vec)
        Xte_vec = scaler.transform(Xte_vec)

    clf = LogisticRegression(max_iter=max_iter, random_state=42, solver="saga", n_jobs=-1)
    clf.fit(Xtr_vec, ytr)

    y_pred = clf.predict(Xte_vec)
    y_proba = clf.predict_proba(Xte_vec)[:, list(clf.classes_).index("positive")]

    print(classification_report(yte, y_pred))
    return clf, vec, evaluate_predictions(yte, y_pred, y_proba)

# BoW униграммы
vec = CountVectorizer(ngram_range=(1, 2), min_df=2)
clf_bow1, vec_bow1, m_bow1 = train_logreg(X_train, X_test, y_train, y_test, vec)
m_bow1


              precision    recall  f1-score   support

    negative       0.76      0.75      0.75     27981
    positive       0.76      0.77      0.76     28728

    accuracy                           0.76     56709
   macro avg       0.76      0.76      0.76     56709
weighted avg       0.76      0.76      0.76     56709



{'accuracy': 0.7570050609250736,
 'f1_macro': 0.756894199711343,
 'f1_pos': 0.762085635359116,
 'roc_auc': np.float64(0.8365177765482765)}

### 7.4 Несколько вариантов признаков (и почему качество может прыгать)

Соберём несколько экспериментов и сравним.  
Чтобы не ждать вечность, мы ограничим словарь (`min_df`, иногда `max_features`). Это нормальная инженерная практика.


In [ ]:
experiments = []

def run_experiment(name, vec, use_scaler=True, max_iter=300):
    print("\n" + "="*80)
    print("🧪", name)
    clf, vec_fitted, metrics = train_logreg(X_train, X_test, y_train, y_test, vec, use_scaler=use_scaler, max_iter=max_iter)
    metrics["name"] = name
    experiments.append(metrics)
    return clf, vec_fitted

# 1) BoW uni
clf1, vec1 = run_experiment("BoW (1,1) слова", CountVectorizer(ngram_range=(1,1), min_df=2))

# 2) BoW tri
clf3, vec3 = run_experiment("BoW (3,3) слова", CountVectorizer(ngram_range=(3,3), min_df=2), max_iter=400)

# 3) TF-IDF uni+bi
clft, vect = run_experiment("TF‑IDF (1,2) слова", TfidfVectorizer(ngram_range=(1,2), min_df=2), max_iter=400)

## 4) Символьные n-граммы (обычно хорошо на «шумном» тексте)
#clfc, vecc = run_experiment("BoW char (3,5)", CountVectorizer(analyzer="char", ngram_range=(3,5), min_df=2), max_iter=10)

pd.DataFrame(experiments).set_index("name").sort_values("f1_macro", ascending=False)



🧪 BoW (1,1) слова
              precision    recall  f1-score   support

    negative       0.74      0.74      0.74     27981
    positive       0.75      0.75      0.75     28728

    accuracy                           0.74     56709
   macro avg       0.74      0.74      0.74     56709
weighted avg       0.74      0.74      0.74     56709


🧪 BoW (3,3) слова
              precision    recall  f1-score   support

    negative       0.74      0.34      0.47     27981
    positive       0.58      0.88      0.70     28728

    accuracy                           0.62     56709
   macro avg       0.66      0.61      0.58     56709
weighted avg       0.66      0.62      0.59     56709


🧪 TF‑IDF (1,2) слова
              precision    recall  f1-score   support

    negative       0.76      0.75      0.75     27981
    positive       0.76      0.77      0.76     28728

    accuracy                           0.76     56709
   macro avg       0.76      0.76      0.76     56709
weighted avg  

,accuracy,f1_macro,f1_pos,roc_auc
name,,,,
"TF‑IDF (1,2) слова",0.758839,0.758704,0.764410,0.840495
"BoW (1,1) слова",0.744961,0.744885,0.749302,0.821488
"BoW (3,3) слова",0.616622,0.584401,0.700120,0.658582


### 7.5 Почему иногда **не** надо убирать пунктуацию 🙂

Сейчас специально добавим пунктуацию как токены (через `word_tokenize`).  
На твитах часто решают смайлики `:)`, `:(`, `;)` и т.п.

⚠️ Это может быть **утечка**: метки могли быть получены именно по смайликам.


In [ ]:
# Эксперимент: токенизация с пунктуацией
tokenizer_ru = lambda s: word_tokenize(s, language="russian")

clf_p, vec_p = run_experiment(
    "BoW (word_tokenize, пунктуация как токены)",
    CountVectorizer(tokenizer=tokenizer_ru, token_pattern=None, min_df=2),
    max_iter=400,
)

pd.DataFrame(experiments).set_index("name").sort_values("f1_macro", ascending=False)



🧪 BoW (word_tokenize, пунктуация как токены)
              precision    recall  f1-score   support

    negative       0.96      0.96      0.96     27981
    positive       0.96      0.96      0.96     28728

    accuracy                           0.96     56709
   macro avg       0.96      0.96      0.96     56709
weighted avg       0.96      0.96      0.96     56709



,accuracy,f1_macro,f1_pos,roc_auc
name,,,,
"BoW (word_tokenize, пунктуация как токены)",0.960059,0.960056,0.960403,0.992760
"BoW (word_tokenize, пунктуация как токены)",0.960059,0.960056,0.960403,0.992760
"TF‑IDF (1,2) слова",0.758839,0.758704,0.764410,0.840495
"BoW (1,1) слова",0.744961,0.744885,0.749302,0.821488
"BoW (3,3) слова",0.616622,0.584401,0.700120,0.658582


### 7.6 Что «самое важное» для модели?

Посмотрим на токены с самыми большими/маленькими коэффициентами логрега.


In [ ]:
len(clf_p.coef_.ravel())

65636

In [ ]:
import numpy as np

feature_names = vec_p.get_feature_names_out()
coefs = clf_p.coef_.ravel()

top_pos = np.argsort(coefs)[-20:][::-1]
top_neg = np.argsort(coefs)[:20]

print("Топ позитивных токенов:")
print(list(zip(feature_names[top_pos], coefs[top_pos])))

print("\nТоп негативных токенов:")
print(list(zip(feature_names[top_neg], coefs[top_neg])))

# Один «супер‑токен»
cool_token = feature_names[top_pos[0]]
print("\n🔥 Самый позитивный токен по коэффициенту:", repr(cool_token))

# Правило без ML: если токен встречается — positive, иначе negative
y_pred_rule = np.array(["positive" if cool_token in t else "negative" for t in X_test])
print("\nМетрики правила:")
print(classification_report(y_test, y_pred_rule))


Топ позитивных токенов:
[(')', np.float64(111.13403875487268)), ('d', np.float64(30.590359384080163)), ('dd', np.float64(9.843567043144883)), ('*', np.float64(9.158061843228108)), ('-d', np.float64(9.039600347263407)), ('ddd', np.float64(8.205959462204957)), ('^_^', np.float64(6.6378503696165545)), ('dddd', np.float64(4.839167499576398)), ('люблю', np.float64(4.245267524455786)), ('х', np.float64(4.1045205802629345)), ('-', np.float64(3.921254980453067)), ('%', np.float64(3.611138737158793)), ('ddddd', np.float64(3.2448119630894947)), ('хорошо', np.float64(2.8833227642646584)), ('xd', np.float64(2.6851081113880353)), ('ахах', np.float64(2.665564837631735)), ('спасибо', np.float64(2.592536270394871)), ('приятно', np.float64(2.48041355298981)), ('dddddd', np.float64(2.304478943330342)), ('ахаха', np.float64(2.2071846254226086))]

Топ негативных токенов:
[('(', np.float64(-93.87161961990589)), ('о_о', np.float64(-9.498150975260733)), ('o_o', np.float64(-6.712595796785696)), ('|', np.float

### 7.7 Таблица экспериментов (итог)

Соберём результаты в одну таблицу.


In [ ]:
results = pd.DataFrame(experiments).set_index("name").sort_values("f1_macro", ascending=False)
results


,accuracy,f1_macro,f1_pos,roc_auc
name,,,,
"BoW (word_tokenize, пунктуация как токены)",0.960059,0.960056,0.960403,0.992760
"BoW (word_tokenize, пунктуация как токены)",0.960059,0.960056,0.960403,0.992760
"TF‑IDF (1,2) слова",0.758839,0.758704,0.764410,0.840495
"BoW (1,1) слова",0.744961,0.744885,0.749302,0.821488
"BoW (3,3) слова",0.616622,0.584401,0.700120,0.658582


💡 **Интерпретация:** если качество внезапно стало почти идеальным — проверьте, нет ли **утечки признаков**.

В этом датасете тональность часто «торчит» в конце твита в виде смайликов `:)`, `:(`, `;)`, и логично, что пунктуация/скобки становятся супер‑важными признаками.


## 8) Регулярные выражения (regex) — чистка и извлечение паттернов

Регулярки помогают:
- убирать «мусор» (URL, @mentions, лишние пробелы)
- вытаскивать нужные шаблоны (email, номера, даты)
- писать кастомную токенизацию

Коротко о спецсимволах:
- `?` — 0 или 1 повторение
- `*` — 0 или больше повторений
- `+` — 1 или больше повторений
- `.` — любой символ
- `[...]` — класс символов
- `\d` — цифра, `\w` — «буквенно‑цифровой», `\s` — пробельный


In [ ]:
import re

# findall: вернуть все совпадения
print(re.findall(r"ab+c.", "abcdefghijkabcabcxabc"))

# split: разбиение по шаблону
print(re.split(r",\s*", "itsy, bitsy, teenie, weenie", maxsplit=2))

# sub: замена
text = "Цена: 123 руб., а потом ещё 456 руб."
print(re.sub(r"\d+", lambda m: str(int(m.group())*2), text))


['abcd', 'abca']
['itsy', 'bitsy', 'teenie, weenie']
Цена: 246 руб., а потом ещё 912 руб.


### Задачка: достать домены из email‑адресов

Дано:

`abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz`

Нужно получить:

`['gmail.com', 'test.in', 'analyticsvidhya.com', 'rest.biz']`


In [ ]:
emails = "abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz"

# Вариант 1: забираем всё после @ до разделителя
domains = re.findall(r"@([A-Za-z0-9\.-]+\.[A-Za-z]{2,})", emails)
print(domains)

# Вариант 2 (чуть строже): домен + зона
domains2 = re.findall(r"@([A-Za-z0-9-]+(?:\.[A-Za-z0-9-]+)+)", emails)
print(domains2)


['gmail.com', 'test.in', 'analyticsvidhya.com', 'rest.biz']
['gmail.com', 'test.in', 'analyticsvidhya.com', 'rest.biz']


### Regex‑токены по правилу

Например, «русское слово» = последовательность букв `А‑Яа‑яё` и дефиса.


In [ ]:
prog = re.compile(r"[А-Яа-яё\-]+")
print(prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё."))


## Итоги и мини‑домашка

Что мы сегодня сделали:
- научились токенизировать текст и понимать, что токенизация бывает «под задачу»
- познакомились со стоп‑словами, лемматизацией и стеммингом
- построили признаки: BoW, n‑граммы, char‑n‑граммы, TF‑IDF
- обучили бейзлайн‑модель и сравнили несколько вариантов признаков
- увидели, что пунктуация может **внезапно** стать ключевой (и почему это бывает утечкой)

Мини‑домашка (на 20–30 минут):
1) Добавьте в `clean_tweet` удаление хэштегов (или, наоборот, сохраните `#тема` как отдельный токен) и сравните метрики.  
2) Ограничьте словарь `max_features=50000` и посмотрите, что будет со временем и качеством.  
3) Попробуйте `LinearSVC` вместо логрега и добавьте в таблицу экспериментов.

Если хотите — в следующем семинаре логично переходить к эмбеддингам (word2vec/fastText) и более «умным» моделям.
